In [ ]:
import numpy as np

In [ ]:
'''
regardless of matched filter or not, lots of different ways to do a log likelihood statistic

two different sets of assumptions
1. what we expect the data to look like if nothing is going on -> everything is drawn to some mean from historical data, and how is the data distributed (poisson, gaussian, binary)
2. how things change for affected subset, q: coefficient. how different is the alternative hypothesis from the null.

matched filter part:
find a subset S and finter theta. theta is constrained to lie in some filter set. penalty based on how different s is from theta.
If something in s and not in theta or vice versa then penalised. Amount of penalty is delta.

search filter set and find 1-nearest neighbour according to L1 metric.

using tree for filter set search:
1. how do you learn the tree to begin with - multi-criterion decision tree. leaves of tree contain one or more filters. split on element contained or not.
2. how do you implement search - 
in each internal node we need to know how many filters in each branch, and for each data element how many filters its included in.
    you can either follow the branch conforming to your subset, or follow the other branch if you believe it will bring you to a closer filter.
    if that other branch has a lot of filters in it, or if the filters are on average closer to your subset.
'''

In [ ]:
'''
HOW WE'RE COMPUTING Q INTERVALS

for normal penalised scan, we have p value for each data point.
for this p value we can get a qmin and qmax for each data point. interval for which data point makes positive contribution to the score.

call this twice: one with 0 penalty, and one with score - delta (penalty of delta)
    this gives us 4 points, q_{min,unpenalized}, q_{min,penalized}, q_{max,penalized}, q_{max,unpenalized}_i for each s_i

take all of the distinct values given and sort them. this gives you 4N-1 intervals, each interval gives you a different combination of points.

we also have positive or negative direction (q > 1 or q < 1), so we also restrict the set of q values in one of these directions.

WEIGHTS

Conditioned on a given q:
LLR_i > Delta -> weight = Delta, element in initial subset
0 < LLR_i < Delta -> weight = LLR_i, element in initial subset
-Delta < LLR_i < 0 -> weight = LLR_i, element not in initial subset
LLR_i < -Delta -> weight = Delta, element not in initial subset
'''

In [ ]:
'''
WHAT TO DO ONCE WE FIND AN OPTIMAL FILTER

if we were able to find a filter which minimises L1 distance to a subset, at a given q value, we know what the best subset is for that filter.
subset can change because of q value:
- data points are always included, always excluded or only included if included in the filter. so once we get the filter we might change our subset a bit.
- once we have this new subset we compute qmle, add up all the counts, add up all the baselines, take the ratio. qmle is a different value from that q interval that we started out with.
'''

In [ ]:
S = []

q_values = []
for i in range(len(S)):
    q_values.append()

'''
Compute (q_{min,unpenalized}, q_{min,penalized}, q_{max,penalized}, q_{max,unpenalized})_i for each s_i
Collect all distinct q_{min} and q_{max} values, sort them, and form ranges (no more than 4N)
'''

q_ranges = [(0, 1)]

# for q_range in q_ranges:
#     (q_min, q_max) = q_range
#     q_temp = (q_min+q_max)/2
#     converged = False
#     while not converged:
#         # computing LLR for each s_i? I thought LLR is calculated for the whole subset.
#         # x_i*log(q) + μ_i*(1 - q)
#         # # (log Pr(xi | qμi) − log Pr(xi | μi)) is it this?
#         # # log(p(xi | Dist1)/p(xi | Dist0)) or this? what is Dist?
#         LLRs = [compute_LLR(s, q_temp) for s in S]
#         S_initial = [int(LLR_i > 0) for LLR_i in LLRs]
#         F_max = sum([max(0, LLR_i) for LLR_i in LLRs])
#         # Define weighted L1 distance metric D and find filter in filter set which minimises D, use tree?
#         F = F_max = D(S_initial, theta)

def compute_LLR(c_i, b_i, q):
    return c_i*np.log(q) + b_i*(1 - q)

for q_range in q_ranges:
    (q_min, q_max) = q_range
    q_temp = (q_min+q_max)/2
    converged = False
    while not converged:
        # computing LLR for each s_i? I thought LLR is calculated for the whole subset.
        # B = baselines
        LLRs = [compute_LLR(s, b, q_temp) for s, b in zip(S, B)]
        weights = []
        S_initial = []
        F_max = 0
        for LLR_i in LLRs:
            S_initial.append(int(LLR_i > 0))
            weights.append(LLR_i) if abs(LLR_i) < Delta else weights.append(Delta)
            F_max += max(0, LLR_i)
        # Define weighted L1 distance metric D and find filter in filter set which minimises D, use tree?
        '''
        This is where the tree search goes
        '''
        F = F_max = D(S_initial, theta)

'''
For each q range (q_{min,range}, q_{max,range}):
    q_{temp} = (q_{min,range} + q_{max,range}) / 2
    converged = False
    While not converged do:
        Compute LLR_i for each s_i at the given q_{temp}
        Define initial subset S: S_i = 1\{ LLR_i > 0 \}
        Define maximum possible score: F_{max} = \sum_i \max(0, LLR_i)
        Define weighted L1 distance metric: D(S, \theta) = \sum_i \min(|LLR_i|,\Delta) 1\{S_i \ne \theta_i \}
        Find \theta \in \Theta that (approximately) minimizes D(S, \theta)
        Compute score: F = F_{max} - D(S, \theta)
        Compute new S: S_i = 1\{ LLR_i > \Delta(1-2\theta_i) \}
        Compute new q_{mle} of S
        If (q_{mle} == q_{temp}):
            converged = True
        else:
            q_{temp} = q_{mle}
'''

In [ ]:
class Node:
    '''
    Multi-criterion decision tree to search the filter set.
    '''

    def __init__(self, split_element):
        self.split_element = split_element
        self.num_filters = 0
        self.element_counts = 0
        self.left = None
        self.right = None

    # information gain for split element

In [32]:
import pandas as pd
import numpy as np

data = {'s1':[1,0,0,1,0], 's2':[1,1,0,1,1], 's3':[0,1,1,1,1], 's4':[0,0,1,0,1]}
df = pd.DataFrame(data)

df.head()

,s1,s2,s3,s4
0,1,1,0,0
1,0,1,1,0
2,0,0,1,1
3,1,1,1,0
4,0,1,1,1


In [131]:
df[df['s1'] == 1].drop(['s1'], axis=1)

,s2,s3,s4
0,1,0,0
3,1,1,0


In [122]:
def entropy(data):
    size = data.shape[0]
    props = [((data[col] == 0).sum(), (data[col] == 1).sum()) for col in data.columns]
    ent = -sum([num_0/size*np.log2(num_0/size) + (num_1/size*np.log2(num_1/size)) for (num_0, num_1) in props if num_0*num_1 > 0]) # if either prop_0 or prop_1 = 0 then entropy = 0
    return ent

In [123]:
print(entropy(df))

3.385757378684062


In [125]:
def info_gains(data):
    size = data.shape[0]
    entropy_before = entropy(data)
    info_gains = []
    for col in data.columns:
        left, right = data[data[col] == 0], data[data[col] == 1]
        left_size, right_size = left.shape[0], right.shape[0]
        entropy_after = (left_size/size)*entropy(left) + (right_size/size)*entropy(right)
        info_gains.append(entropy_before - entropy_after)
    return info_gains

In [128]:
df.columns[0]

's1'

In [126]:
def build_tree_rec(data):
    num_filters = data.shape[0]
    element_counts = [(data[col] == 1).sum() for col in data.columns]
    max_ig_element = data.columns[np.argmax(info_gains(data))[0]]
    tree = Node(max_ig_element, num_filters, element_counts)
    tree.left = build_tree_rec(data[data[max_ig_element] == 0])
    tree.right = build_tree_rec(data[data[max_ig_element] == 1])
    '''
    PROBLEM: the ID3 algorithm removes the element we just made a split on for the next split.
             but we want to keep element counts at all nodes. so we need to include this element.
    '''

    '''
    Extra check: if nothing is splitting the data then stop expanding
    '''

    '''
    Bound depth of decision tree: root n
    '''

[1.8838023778186743, 1.2877123795494492, 1.2877123795494492, 1.8838023778186743]


In [ ]:
'''
Stack for storing the other branches
For now: if rules give different branches then pick one and push the other to the stack.
Later: only push other to stack if it is better by some margin.
'''

'''
Traversing the tree: elements have weights
If rules give different branches then pick one: we should be more likely to pick the branch
that contains the element, if the element has a high weight.
- At each node: calculate distance to estimate filter (from summary statistics) in each branch
'''

'''
Traversing the tree:
assuming IID, we can calculate a mean and s.d. weighted distance to any filter for each of the two branches.
take 20 draws from this gaussian distribution, take the lowest of these draws.
'''

In [ ]:
'''
tree traversal: take in subset and weights for each element
'''

In [ ]:
'''
OTHER IDEA

Binary decision diagrams
Ways of representing sets of subsets
Variant: Zero-suppressed binary decision diagrams - only the things that correspond to actual subsets.
https://en.wikipedia.org/wiki/Zero-suppressed_decision_diagram
'''